In [12]:

import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


In [13]:

# Load the dataset
df = pd.read_csv("news.csv")
df.dropna(inplace=True)
print(df['label'].value_counts())  # Check label balance


label
REAL    3171
FAKE    3164
Name: count, dtype: int64


In [14]:

# Encode labels: FAKE = 1, REAL = 0
df['label'] = df['label'].map({'FAKE': 1, 'REAL': 0})


In [15]:

X = df['text']
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [16]:

tfidf = TfidfVectorizer(stop_words='english', max_df=0.7)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)


In [17]:

model = PassiveAggressiveClassifier(max_iter=1000, class_weight='balanced')
model.fit(X_train_tfidf, y_train)


,C,1.0
,fit_intercept,True
,max_iter,1000
,tol,0.001
,early_stopping,False
,validation_fraction,0.1
,n_iter_no_change,5
,shuffle,True
,verbose,0
,loss,'hinge'
,n_jobs,None


In [18]:

y_pred = model.predict(X_test_tfidf)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.9337016574585635
Confusion Matrix:
 [[596  43]
 [ 41 587]]
Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.93      0.93       639
           1       0.93      0.93      0.93       628

    accuracy                           0.93      1267
   macro avg       0.93      0.93      0.93      1267
weighted avg       0.93      0.93      0.93      1267



In [19]:

with open("Fake_NEWS_model11.pkl", "wb") as f:
    pickle.dump(model, f)
with open("tf_idf11.pkl", "wb") as f:
    pickle.dump(tfidf, f)


In [20]:
import pickle

# Load model and vectorizer
with open("Fake_NEWS_model11.pkl", "rb") as f:
    model = pickle.load(f)
with open("tf_idf11.pkl", "rb") as f:
    tfidf = pickle.load(f)

# New Fake News Examples
fake_news_examples = [
    "Government Launches Mind Control Chips in COVID-19 Vaccines, Leaked Documents Reveal",
    "Elon Musk Says Mars Is Already Populated, Plans to Visit Colony by 2026",
    "Drinking Bleach Can Cure Cancer, Claims Viral Facebook Post",
    "Apple to Release iPhone That Can Read Your Thoughts, Claims Insider",
    "Area 51 Scientists Admit Working with Alien Technology to Control Weather"
]

# New Real News Examples
real_news_examples = [
    "Apple Unveils iOS 18 with AI Features at WWDC 2025",
    "ISRO Plans First Human Spaceflight Mission by 2026, Says Chairman",
    "Global Markets Rally After Positive Economic Forecast from IMF",
    "Scientists Develop New Technique to Recycle Ocean Plastic into Fuel",
    "World Health Organization Launches Global Preparedness Initiative for Future Pandemics"
]

# Combine all examples
all_examples = fake_news_examples + real_news_examples

# Vectorize and predict
vectors = tfidf.transform(all_examples)
predictions = model.predict(vectors)

# Output results
for text, pred in zip(all_examples, predictions):
    print(f"Text: {text}")
    print("Prediction:", "FAKE" if pred == 1 else "REAL")
    print("-" * 60)


Text: Government Launches Mind Control Chips in COVID-19 Vaccines, Leaked Documents Reveal
Prediction: FAKE
------------------------------------------------------------
Text: Elon Musk Says Mars Is Already Populated, Plans to Visit Colony by 2026
Prediction: FAKE
------------------------------------------------------------
Text: Drinking Bleach Can Cure Cancer, Claims Viral Facebook Post
Prediction: FAKE
------------------------------------------------------------
Text: Apple to Release iPhone That Can Read Your Thoughts, Claims Insider
Prediction: FAKE
------------------------------------------------------------
Text: Area 51 Scientists Admit Working with Alien Technology to Control Weather
Prediction: FAKE
------------------------------------------------------------
Text: Apple Unveils iOS 18 with AI Features at WWDC 2025
Prediction: FAKE
------------------------------------------------------------
Text: ISRO Plans First Human Spaceflight Mission by 2026, Says Chairman
Prediction: RE